In [49]:
from azureml.core import Workspace, Experiment, Dataset

ws = Workspace.get(name="quick-starts-ws-121940")
exp = Experiment(workspace=ws, name="udacity-project-demo")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-121940
Azure region: southcentralus
Subscription id: 3ec87f2b-33cf-4aed-b36f-750175813524
Resource group: aml-quickstarts-121940


In [50]:
from azureml.core.compute import ComputeTarget, AmlCompute
cpu_cluster_name= "udacity-project-demo"
try:
    compute_target= ComputeTarget(workspace=ws, name= cpu_cluster_name)
    print('found')       
except ComputeTargetException:
    compute_config= AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes= 4)
    compute_target= ComputeTarget.create(ws, cpu_cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True)

found

Jobrunning
Running


In [51]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import choice, loguniform
import os
ps = RandomParameterSampling({
          '--batch-size': choice (25, 50, 100),
          '--first-layer-neurons': choice (10, 50, 200, 300, 500),
          '--second-layer-neurons': choice (10, 50, 200, 500),
          '--learning-rate': loguniform (-6, -1)
})
policy = BanditPolicy(evaluation_interval= 2, slack_factor= 0.1)
if "training" not in os.listdir():
    os.mkdir("./training")
est = SKLearn(source_directory='./', compute_target= cpu_cluster_name, entry_script= "train.py")
hyperdrive_config = HyperDriveConfig(estimator=est, hyperparameter_sampling=ps, policy=policy, primary_metric_name='Accuracy', primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                    max_total_runs=20, max_concurrent_runs=4)

In [52]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hdr=exp.submit(config=hyperdrive_config)
RunDetails(hdr).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [53]:
import joblib
best_run = hdr.get_best_run_by_primary_metric()

In [54]:
from azureml.core import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
web_path ='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
dataset = Dataset.Tabular.from_delimited_files(path= web_path)
df = dataset.to_pandas_dataframe()

In [55]:
from azureml.core import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from train import clean_data
x, y = clean_data(df)

NameError: name 'clean_data' is not defined

In [ ]:
from azureml.train.automl import AutoMLConfig
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task= 'classification',
    primary_metric='AUC_Weighted',
    training_data= dataset,
    label_column_name= 'job',
    n_cross_validations= 5)

In [ ]:
# Submit your automl run
automl_run= exp.submit(config= automl_config, show_output= True)

In [ ]:
# Retrieve and save your best automl model.
best_run = automl_run.get_best_run_by_primary_metric()